In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#done for both test and train
import os
import numpy as np

# Path to the base directory
base_dir = '/content/drive/My Drive/vgg_test_features'

# Path to the output directory
output_base_dir = '/content/drive/My Drive/vgg_test_features_concatenated'

# Main folders names
main_folders = ['Right', 'Left', 'Front']

# Get the names of subfolders (assuming the same structure in all main folders)
subfolders = os.listdir(os.path.join(base_dir, main_folders[0]))

# Function to load and concatenate .npy files from identical paths across different main folders
def concatenate_npy_files(base_dir, main_folders, subfolder, filename):
    arrays = []
    paths = []
    for folder in main_folders:
        file_path = os.path.join(base_dir, folder, subfolder, filename)
        arrays.append(np.load(file_path))
        paths.append(file_path)
    concatenated_array = np.concatenate(arrays, axis=0)
    return concatenated_array, paths, [array.shape for array in arrays]

# Create output directory and subfolders if they don't exist
os.makedirs(output_base_dir, exist_ok=True)
for subfolder in subfolders:
    os.makedirs(os.path.join(output_base_dir, subfolder), exist_ok=True)

# Iterate through subfolders and files to concatenate
for subfolder in subfolders:
    file_names = os.listdir(os.path.join(base_dir, main_folders[0], subfolder))
    for filename in file_names:
        concatenated_array, paths, shapes = concatenate_npy_files(base_dir, main_folders, subfolder, filename)

        # Print the paths and initial dimensions
        print(f"Concatenating the following files for {subfolder}/{filename}:")
        for path, shape in zip(paths, shapes):
            print(f" - {path}, shape: {shape}")

        # Print the final dimension
        print(f"Final concatenated shape: {concatenated_array.shape}")

        # Save the concatenated array in the appropriate subfolder within the output directory
        save_path = os.path.join(output_base_dir, subfolder, filename)
        np.save(save_path, concatenated_array)
        print(f"Saved concatenated array at {save_path}\n")

print("All files concatenated and saved successfully.")


Concatenating the following files for Straight/ccdb92f0-d17d-4559-8055-55d9f9e1783e.npy:
 - /content/drive/My Drive/vgg_test_features/Right/Straight/ccdb92f0-d17d-4559-8055-55d9f9e1783e.npy, shape: (360, 1, 512)
 - /content/drive/My Drive/vgg_test_features/Left/Straight/ccdb92f0-d17d-4559-8055-55d9f9e1783e.npy, shape: (360, 1, 512)
 - /content/drive/My Drive/vgg_test_features/Front/Straight/ccdb92f0-d17d-4559-8055-55d9f9e1783e.npy, shape: (360, 1, 512)
Final concatenated shape: (1080, 1, 512)
Saved concatenated array at /content/drive/My Drive/vgg_test_features_concatenated/Straight/ccdb92f0-d17d-4559-8055-55d9f9e1783e.npy

Concatenating the following files for Straight/5a0b566e-a6e1-4379-aaf6-c12603889de1.npy:
 - /content/drive/My Drive/vgg_test_features/Right/Straight/5a0b566e-a6e1-4379-aaf6-c12603889de1.npy, shape: (450, 1, 512)
 - /content/drive/My Drive/vgg_test_features/Left/Straight/5a0b566e-a6e1-4379-aaf6-c12603889de1.npy, shape: (450, 1, 512)
 - /content/drive/My Drive/vgg_tes

In [ ]:
# #channel-wise fusion
# import os
# import numpy as np

# # Path to the base directory
# base_dir = '/content/drive/My Drive/vgg_train_features'

# # Path to the output directory
# output_base_dir = '/content/drive/My Drive/vgg_train_features_cw'

# # Main folders names
# main_folders = ['Right', 'Left', 'Front']

# # Get the names of subfolders (assuming the same structure in all main folders)
# subfolders = os.listdir(os.path.join(base_dir, main_folders[0]))

# # Function to load and stack .npy files from identical paths across different main folders
# def stack_npy_files(base_dir, main_folders, subfolder, filename):
#     arrays = []
#     paths = []
#     for folder in main_folders:
#         file_path = os.path.join(base_dir, folder, subfolder, filename)
#         arrays.append(np.load(file_path))
#         paths.append(file_path)
#     stacked_array = np.stack(arrays, axis=-1)  # Stack along a new axis (channel-wise)
#     return stacked_array, paths, [array.shape for array in arrays]

# # Create output directory and subfolders if they don't exist
# os.makedirs(output_base_dir, exist_ok=True)
# for subfolder in subfolders:
#     os.makedirs(os.path.join(output_base_dir, subfolder), exist_ok=True)

# # Iterate through subfolders and files to stack
# for subfolder in subfolders:
#     file_names = os.listdir(os.path.join(base_dir, main_folders[0], subfolder))
#     for filename in file_names:
#         stacked_array, paths, shapes = stack_npy_files(base_dir, main_folders, subfolder, filename)

#         # Print the paths and initial dimensions
#         print(f"Stacking the following files for {subfolder}/{filename}:")
#         for path, shape in zip(paths, shapes):
#             print(f" - {path}, shape: {shape}")

#         # Print the final dimension
#         print(f"Final stacked shape: {stacked_array.shape}")

#         # Save the stacked array in the appropriate subfolder within the output directory
#         save_path = os.path.join(output_base_dir, subfolder, filename)
#         np.save(save_path, stacked_array)
#         print(f"Saved stacked array at {save_path}\n")

# print("All files stacked and saved successfully.")


In [7]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-03bqdhu3
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-03bqdhu3
  Resolved https://github.com/tensorflow/docs to commit 75b2672b5bed8ca0995663536db84bd9a39b8896
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2024.5.3.31743-py3-none-any.whl size=182531 sha256=97ca38636a375d07cb86308f84e1598c9530b954f7313206247425b0aadb1b2f
  Stored in directory: /tmp/pip-ephem-wheel-cache-dwg1io6c/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [11]:
dataset_path = os.listdir('/content/drive/MyDrive/vgg_train_features_concatenated')

label_types = os.listdir('/content/drive/MyDrive/vgg_train_features_concatenated')
print (label_types)

['Left Lane Change', 'Right Turn', 'Straight', 'Right Lane Change', 'Slow-Stop', 'Left Turn']


In [12]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/vgg_train_features_concatenated' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/vgg_train_features_concatenated' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

                tag                                         video_name
0  Left Lane Change  /content/drive/MyDrive/vgg_train_features_conc...
1  Left Lane Change  /content/drive/MyDrive/vgg_train_features_conc...
2  Left Lane Change  /content/drive/MyDrive/vgg_train_features_conc...
3  Left Lane Change  /content/drive/MyDrive/vgg_train_features_conc...
4  Left Lane Change  /content/drive/MyDrive/vgg_train_features_conc...
           tag                                         video_name
402  Left Turn  /content/drive/MyDrive/vgg_train_features_conc...
403  Left Turn  /content/drive/MyDrive/vgg_train_features_conc...
404  Left Turn  /content/drive/MyDrive/vgg_train_features_conc...
405  Left Turn  /content/drive/MyDrive/vgg_train_features_conc...
406  Left Turn  /content/drive/MyDrive/vgg_train_features_conc...


In [13]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('/content/drive/MyDrive/train_v.csv')

In [14]:
dataset_path = os.listdir('/content/drive/MyDrive/vgg_test_features_concatenated')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/vgg_test_features_concatenated')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/vgg_test_features_concatenated' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/vgg_test_features_concatenated' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('/content/drive/MyDrive/test_v.csv')

['Straight', 'Slow-Stop', 'Right Turn', 'Right Lane Change', 'Left Turn', 'Left Lane Change']
Types of activities found:  6
        tag                                         video_name
0  Straight  /content/drive/MyDrive/vgg_test_features_conca...
1  Straight  /content/drive/MyDrive/vgg_test_features_conca...
2  Straight  /content/drive/MyDrive/vgg_test_features_conca...
3  Straight  /content/drive/MyDrive/vgg_test_features_conca...
4  Straight  /content/drive/MyDrive/vgg_test_features_conca...
                 tag                                         video_name
91  Left Lane Change  /content/drive/MyDrive/vgg_test_features_conca...
92  Left Lane Change  /content/drive/MyDrive/vgg_test_features_conca...
93  Left Lane Change  /content/drive/MyDrive/vgg_test_features_conca...
94  Left Lane Change  /content/drive/MyDrive/vgg_test_features_conca...
95  Left Lane Change  /content/drive/MyDrive/vgg_test_features_conca...


In [21]:
train_df = pd.read_csv("/content/drive/MyDrive/train_v.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test_v.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 407
Total videos for testing: 96


,Unnamed: 0,video_name,tag
42,42,/content/drive/MyDrive/vgg_train_features_conc...,Right Turn
342,342,/content/drive/MyDrive/vgg_train_features_conc...,Left Turn
283,283,/content/drive/MyDrive/vgg_train_features_conc...,Left Turn
296,296,/content/drive/MyDrive/vgg_train_features_conc...,Left Turn
233,233,/content/drive/MyDrive/vgg_train_features_conc...,Right Lane Change
246,246,/content/drive/MyDrive/vgg_train_features_conc...,Slow-Stop
256,256,/content/drive/MyDrive/vgg_train_features_conc...,Slow-Stop
1,1,/content/drive/MyDrive/vgg_train_features_conc...,Left Lane Change
94,94,/content/drive/MyDrive/vgg_train_features_conc...,Right Turn
402,402,/content/drive/MyDrive/vgg_train_features_conc...,Left Turn


In [22]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
       [3],
    

In [23]:
#hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 512

In [24]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

In [25]:
EPOCHS = 30

In [27]:
# Define the function to prepare all videos
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values

    # Convert class labels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # For each video
    for idx, path in enumerate(video_paths):
        # Load concatenated features from the appropriate file
        features_path = os.path.join(root_dir, path)
        features = np.load(features_path, allow_pickle = True)
        features = np.squeeze(features)

        # Extract the features and masks
        num_frames = min(MAX_SEQ_LENGTH, features.shape[0])
        frame_features[idx, :num_frames, :] = features[:num_frames, :]
        frame_masks[idx, :num_frames] = 1  # 1 = not masked, 0 = masked

    return (frame_features, frame_masks), labels


# Define the function to run the experiment
def run_experiment(train_data, train_labels, test_data, test_labels):
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


# Load and prepare training and testing data
train_data, train_labels = prepare_all_videos(train_df, "/content/drive/MyDrive/vgg_train_features_concatenated")
test_data, test_labels = prepare_all_videos(test_df, "/content/drive/MyDrive/vgg_test_features_concatenated")

# Run the experiment
_, sequence_model = run_experiment(train_data, train_labels, test_data, test_labels)


Epoch 1/30
8/9 [=========================>....] - ETA: 0s - loss: 1.9487 - accuracy: 0.1250
Epoch 1: val_loss improved from inf to 1.53085, saving model to ./tmp/video_classifier
9/9 [==============================] - 14s 446ms/step - loss: 1.9476 - accuracy: 0.1127 - val_loss: 1.5308 - val_accuracy: 0.6585
Epoch 2/30
9/9 [==============================] - ETA: 0s - loss: 1.8396 - accuracy: 0.0739
Epoch 2: val_loss improved from 1.53085 to 1.50204, saving model to ./tmp/video_classifier
9/9 [==============================] - 1s 61ms/step - loss: 1.8396 - accuracy: 0.0739 - val_loss: 1.5020 - val_accuracy: 0.9593
Epoch 3/30
8/9 [=========================>....] - ETA: 0s - loss: 1.8009 - accuracy: 0.1172
Epoch 3: val_loss did not improve from 1.50204
9/9 [==============================] - 0s 57ms/step - loss: 1.8105 - accuracy: 0.1162 - val_loss: 1.5497 - val_accuracy: 0.6016
Epoch 4/30
9/9 [==============================] - ETA: 0s - loss: 1.7703 - accuracy: 0.1761
Epoch 4: val_loss did